Rebuild street graph
====================
Loads a street graph and reallocates roads space, e.g., into a network of one-way streets.

Prerequisite: Preparing the simplified street graph

In [1]:
import copy

import networkx as nx

import snman, os
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

#CRS_internal = 29119    # for Boston
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

Load street graph


In [6]:
print('Load perimeters')
perimeters_gdf = snman.io.load_perimeters(os.path.join(inputs_path, 'perimeters', 'perimeters.shp'), crs=CRS_internal)

print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    only_active=True,
    #filter_ids=['zrh-1'],
    crs=CRS_internal
)

Load perimeters
Load rebuilding regions


C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
snman.rebuilding.multi_rebuild_regions(
    G, 
    rebuilding_regions_gdf,
    width_attribute=KEY_LANES_DESCRIPTION + '_width_total_m',
    #rebuilding_function=snman.rebuilding.multi_rebuild,
    #given_lanes_function=snman.rebuilding.multi_set_given_lanes,
    public_transit_mode='mandatory_like_existing',
    parking_mode='mandatory_like_existing',
    needed_node_access_function=snman.rebuilding.multi_set_needed_node_access,
    existing_lanes_attribute=KEY_LANES_DESCRIPTION,
    given_lanes_attribute=KEY_GIVEN_LANES_DESCRIPTION,
    target_lanes_attribute=KEY_LANES_DESCRIPTION_AFTER,
    export_L=(os.path.join(export_path, 'L_edges.gpkg'), os.path.join(export_path, 'L_nodes.gpkg')),
    export_H=(os.path.join(export_path, 'H_edges.gpkg'), os.path.join(export_path, 'H_nodes.gpkg')),
    export_when='before_and_after',
    verbose=True
)

rebuilding region zrh-main
include {'1_main_road'}
fix {'4_path', '2_local_road', '0_highway', '3_dead_end', '9_other'}


In [ ]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

print('Nornmalize cycling infrastructure')
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

Add edge costs
--------------
We add the mode-specific cost to the edges so that they can be easily inspected

In [ ]:
print('Add edge costs')
snman.street_graph.add_edge_costs(G, lanes_description=KEY_LANES_DESCRIPTION)
snman.street_graph.add_edge_costs(G, lanes_description=KEY_LANES_DESCRIPTION_AFTER)

Export
------
Saves the resulting datasets to the disk. Use the *snman_detailed.qgz* file to view them in QGIS.

In [ ]:
if 1:
    print('Export network without lanes')
    snman.io.export_street_graph(
        G,
        os.path.join(export_path, 'street_graph_edges.gpkg'),
        os.path.join(export_path, 'street_graph_nodes.gpkg'),
        crs=CRS_for_export
    )

if 1:
    print('Export lane geometries')
    SCALING = 3
    snman.io.export_street_graph_with_lanes(
        G,
        [KEY_LANES_DESCRIPTION, KEY_LANES_DESCRIPTION_AFTER],
        os.path.join(export_path, 'lane_geometries.shp'),
        scaling=SCALING,
        crs=CRS_for_export
    )

if 1:
    print('Export lane graphs')
    L_before = snman.lane_graph.create_lane_graph(G, KEY_LANES_DESCRIPTION)
    L_after = snman.lane_graph.create_lane_graph(G, KEY_LANES_DESCRIPTION_AFTER)
    
    for uvk, data in L_before.edges.items():
        data['_cost_increase_by_removal'] = snman.graph.cost_increase_by_edge_removal(G, *uvk, 'cost_cycling')
    
    snman.io.export_street_graph(
        L_before,
        os.path.join(export_path, 'lane_graph_edges.gpkg'),
        os.path.join(export_path, 'lane_graph_nodes.gpkg'),
        crs=CRS_for_export
    )
    snman.io.export_street_graph(
        L_after,
        os.path.join(export_path, 'lane_graph_after_edges.gpkg'),
        os.path.join(export_path, 'lane_graph_after_nodes.gpkg'),
        crs=CRS_for_export
    )

In [3]:
print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
    #only_active=True,
    filter_ids=['test-98'],
    crs=CRS_internal
)

Load rebuilding regions


In [4]:
snman.rebuilding.multi_rebuild_regions(
    G, 
    rebuilding_regions_gdf,
    width_attribute=KEY_LANES_DESCRIPTION + '_width_total_m',
    #rebuilding_function=lambda L, *x, **kwargs: L,
    #given_lanes_function=snman.rebuilding.multi_set_given_lanes,
    public_transit_mode='mandatory_like_existing',
    parking_mode='mandatory_like_existing',
    needed_node_access_function=snman.rebuilding.multi_set_needed_node_access,
    existing_lanes_attribute=KEY_LANES_DESCRIPTION,
    given_lanes_attribute=KEY_GIVEN_LANES_DESCRIPTION,
    target_lanes_attribute=KEY_LANES_DESCRIPTION_AFTER,
    export_L=(os.path.join(export_path, 'L_edges.gpkg'), os.path.join(export_path, 'L_nodes.gpkg')),
    export_H=(os.path.join(export_path, 'H_edges.gpkg'), os.path.join(export_path, 'H_nodes.gpkg')),
    export_when='before_and_after',
    verbose=True
)

rebuilding region test-98
include {'4_path', '1_main_road', '9_other', '2_local_road', '3_dead_end', '0_highway'}
fix {'4_path', '9_other'}


KeyError: 'grade'

In [ ]:
for uvk, data in G.edges.items():
    data[KEY_LANES_DESCRIPTION_AFTER] = copy.deepcopy(data[KEY_LANES_DESCRIPTION])

In [ ]:
import numpy as np

L = snman.io.load_street_graph(os.path.join(export_path, 'L_edges.gpkg'), os.path.join(export_path, 'L_nodes.gpkg'))

M = L.subgraph(
        [i for i, data in L.nodes.items() if data['needs_access_by_private_cars']]
    )

M = M.edge_subgraph(
    [uvk for uvk, data in M.edges.items() if data.get('cost_private_cars') != np.inf and uvk not in ()]
)

snman.oxc.plot_graph(M)
snman.rebuilding.is_strongly_connected_plus(
    M,
    'cost_private_cars',
    'needs_access_by_private_cars'
)
nx.is_strongly_connected(M)

In [ ]:
scc = nx.strongly_connected_components(M)
list(scc)